In [46]:
import openmm
import openmm.app as app
import openmm.unit as u

pdb_fn = "/home/yppatel/misc/chignolin_idp/combined.pdb"
pdb = app.PDBFile(pdb_fn)
forcefield = app.ForceField("/home/yppatel/misc/chignolin_idp/amber14/protein.ff14SBonlysc.xml", "implicit/gbn2.xml")

system = forcefield.createSystem(pdb.topology, nonbondedMethod = app.forcefield.NoCutoff, constraints = app.forcefield.HBonds)
integrator = openmm.VerletIntegrator(0.002 * u.picoseconds)

platform_name = "CUDA"

platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name == "CUDA":
    prop = dict(CudaPrecision="mixed", DeviceIndex=f"0")
    simulator = app.Simulation(pdb.topology, system, integrator, platform, prop)
else:
    simulator = app.Simulation(pdb.topology, system, integrator, platform)

In [47]:
num_replicates = 2
atoms_per_replicate = system.getNumParticles() // num_replicates

for atom_idx1 in range(atoms_per_replicate):
    for replicate in range(1, num_replicates):
        for atom_idx2 in range(atoms_per_replicate):
            replicate_idx2 = replicate * atoms_per_replicate + atom_idx2
            system.getForces()[2].addException(atom_idx1, replicate_idx2, 0, 0, 0)

In [59]:
pdb.topology

<Topology; 2 chains, 20 residues, 276 atoms, 282 bonds>

In [52]:
simulator.context.setPositions(pdb.positions)

In [51]:
openmm.LocalEnergyMinimizer.minimize(simulator.context)

Calling single minimize()
CUDA test
CPU test


OpenMMException: Particle coordinate is NaN.  For more information, see https://github.com/openmm/openmm/wiki/Frequently-Asked-Questions#nan

In [50]:
energy_kj = simulator.context.getState(getEnergy=True).getPotentialEnergy()
energy_kcal = energy_kj.in_units_of(u.kilocalories_per_mole) # match RDKit/MMFF convention
energy_kcal._value

CUDA test


nan